In [1]:
SEED = 0
cores = 16

from numpy.random import seed
seed(SEED)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import Counter
from xgboost import XGBClassifier
from sklearn.preprocessing import KBinsDiscretizer
from explainerdashboard import ClassifierExplainer, ExplainerDashboard

# version check
import sys
print("python version")
print(sys.version)
import sklearn
print("sklearn version = ", sklearn.__version__)
print("numpy version = ", np.__version__)
print("pandas version =", pd.__version__)

In /home/danssa/anaconda3/envs/ed/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/danssa/anaconda3/envs/ed/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/danssa/anaconda3/envs/ed/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/danssa/anaconda3/envs/ed/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/danssa/anaconda3/en

python version
3.7.10 (default, Feb 26 2021, 18:47:35) 
[GCC 7.3.0]
sklearn version =  0.24.1
numpy version =  1.20.2
pandas version = 1.2.3


In [2]:
df = pd.read_csv("~/proj_ua/data/ua_full_10types_new.v7.csv", dtype={'id':str})
df2 = df[(df['from']=="cha")]
df2['eGFR_ab45'] = np.where(df2['eGFR_ckd']<45,1,0)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
#3group age split  

##step 1 finding edge value
abnormal_disc = df2.query('eGFR_ab45==1').loc[:,'age']
abnormal_disc = pd.DataFrame(abnormal_disc)

est = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='kmeans')
est.fit(abnormal_disc)

ab_disc = est.transform(abnormal_disc).astype('float')
print("edges : ", est.bin_edges_[0])

age0_edge = est.bin_edges_[0][1]
age1_edge = est.bin_edges_[0][2]
age2_edge = est.bin_edges_[0][3]
print('age0_edge:', age0_edge, '\nage1_edge:', age1_edge, '\nage2_edge:', age2_edge)

abnormal_disc['level'] = abnormal_disc.apply(lambda x : 0 if x['age']<age0_edge else 1 if x['age']<age1_edge else 2, axis=1)
print('age group:\n',abnormal_disc['level'].value_counts().sort_index())

edges :  [18.         50.91879412 69.93915802 90.        ]
age0_edge: 50.91879411967486 
age1_edge: 69.9391580177803 
age2_edge: 90.0
age group:
 0     723
1    1966
2    3234
Name: level, dtype: int64


In [4]:
##make 3group by age
df3 = df2.copy()

##age cut set up
df3['level'] = df3.apply(lambda x : 0 if x['age']<age0_edge else 1 if x['age']<age1_edge else 2, axis=1)
print(df3['level'].value_counts())

0    132883
1     56994
2     20985
Name: level, dtype: int64


In [5]:
##age0 group
X_age0 = df3[df3['level']==0]
y_age0 = X_age0['eGFR_ab45'].astype("int64")

print('\nage0', X_age0['age'].min(), X_age0['age'].max())
print("total cases = %d" %X_age0.shape[0])
print("total abnormal function of kidney = %d" %sum(y_age0), "\nprevalence =", sum(y_age0)/len(y_age0)*100)

X_train0, X_test0, y_train0, y_test0 = train_test_split(X_age0, y_age0, test_size=0.2, stratify=y_age0, random_state=SEED)
print("train0 : %d" % sum(y_train0), "test0 : %d" % sum(y_test0))

##age1 group
X_age1 = df3[df3['level']==1]
y_age1 = X_age1['eGFR_ab45']

print('\nage1', X_age1['age'].min(), X_age1['age'].max())
print("total cases = %d" %X_age1.shape[0])
print("total abnormal function of kidney = %d" %sum(y_age1), "\nprevalence =", sum(y_age1)/len(y_age1)*100)

X_train1, X_test1, y_train1, y_test1 = train_test_split(X_age1, y_age1, test_size=0.2, stratify=y_age1, random_state=SEED)
print("train1 : %d" % sum(y_train1), "test0 : %d" % sum(y_test1))

##age2 group
X_age2 = df3[df3['level']==2]
y_age2 = X_age2['eGFR_ab45']

print('\nage2', X_age2['age'].min(), X_age2['age'].max())
print("total cases = %d" %X_age2.shape[0])
print("total abnormal function of kidney = %d" %sum(y_age2), "\nprevalence =", sum(y_age2)/len(y_age2)*100)

X_train2, X_test2, y_train2, y_test2 = train_test_split(X_age2, y_age2, test_size=0.2, stratify=y_age2, random_state=SEED)
print("train2 : %d" % sum(y_train2), "test2 : %d" % sum(y_test2))


age0 18 50
total cases = 132883
total abnormal function of kidney = 723 
prevalence = 0.5440876560583371
train0 : 578 test0 : 145

age1 51 69
total cases = 56994
total abnormal function of kidney = 1966 
prevalence = 3.4494859107976277
train1 : 1573 test0 : 393

age2 70 90
total cases = 20985
total abnormal function of kidney = 3234 
prevalence = 15.411007862759114
train2 : 2587 test2 : 647


In [6]:
##concat both trainset and testset
X_train = pd.concat([X_train0, X_train1, X_train2])
y_train = pd.concat([y_train0, y_train1, y_train2])

X_test = pd.concat([X_test0, X_test1, X_test2])
y_test = pd.concat([y_test0, y_test1, y_test2])

print("total cases = %d" % (X_train.shape[0] + X_test.shape[0]))
print("total abnormal function of kidney = %d" % (sum(y_train) + sum(y_test)), "\nprevalence =", (sum(y_train)+sum(y_test))/(len(y_train)+len(y_test))*100)

total cases = 210862
total abnormal function of kidney = 5923 
prevalence = 2.8089461353871252


In [7]:
X_train_features = X_train.loc[:, 'male':'leucocyte']

print('%d train cases, %d variables' % (X_train_features.shape[0], X_train_features.shape[1]))
print('%d test cases'%X_test.shape[0])

168689 train cases, 12 variables
42173 test cases


In [8]:
#standardization

scaler = StandardScaler()
std_cols=['age','he_uph','he_usg']
std_df=X_train_features[std_cols]

X_train_features[std_cols]=scaler.fit_transform(std_df)
X_train_features.describe()

,male,age,he_uph,he_unitr,he_usg,he_upro,he_uglu,he_uket,he_ubil,he_ubld,he_uro,leucocyte
count,168689.000000,1.686890e+05,1.686890e+05,168689.000000,1.686890e+05,168689.000000,168689.000000,168689.000000,168689.000000,168689.000000,168689.000000,168689.000000
mean,0.421332,-5.391547e-17,-1.145704e-16,0.019835,1.648803e-14,0.326749,0.198288,0.201299,0.037412,0.772404,0.126772,0.511717
std,0.493774,1.000003e+00,1.000003e+00,0.139435,1.000003e+00,0.763357,0.821801,0.664694,0.319771,1.262495,0.528855,1.021307
min,0.000000,-1.851626e+00,-1.472512e+00,0.000000,-1.721268e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,-7.550769e-01,-8.614876e-01,0.000000,-1.089549e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,-1.745511e-01,-2.504637e-01,0.000000,1.738895e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,7.284891e-01,9.715841e-01,0.000000,8.056085e-01,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,1.000000,2.792581e+00,3.415680e+00,1.000000,1.437328e+00,5.000000,5.000000,5.000000,4.000000,5.000000,4.000000,4.000000


In [9]:
#testset
X_test_features = X_test.loc[:, 'male':'leucocyte']
std_df_test=X_test_features[std_cols]
X_test_features[std_cols]=scaler.transform(std_df_test)

X_test_features.describe()

,male,age,he_uph,he_unitr,he_usg,he_upro,he_uglu,he_uket,he_ubil,he_ubld,he_uro,leucocyte
count,42173.000000,42173.000000,42173.000000,42173.000000,42173.000000,42173.000000,42173.000000,42173.000000,42173.000000,42173.000000,42173.000000,42173.000000
mean,0.423992,-0.002073,-0.010505,0.020345,0.006452,0.326892,0.196073,0.203614,0.036872,0.774856,0.124392,0.520120
std,0.494195,0.999623,1.001929,0.141178,0.998775,0.765950,0.818686,0.677874,0.318040,1.268394,0.526078,1.028755
min,0.000000,-1.851626,-1.472512,0.000000,-1.721268,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,-0.755077,-0.861488,0.000000,-1.089549,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,-0.174551,-0.250464,0.000000,0.173889,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.728489,0.971584,0.000000,0.805609,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,1.000000,2.792581,3.415680,1.000000,1.437328,5.000000,5.000000,5.000000,4.000000,4.000000,4.000000,4.000000


In [10]:
#model : ua_cha45_10types.v4
model = XGBClassifier(colsample_bytree=0.8, gamma=0.0, learning_rate=0.01, max_depth=4, 
                          min_child_weight=10, n_estimators=500, objective="binary:logistic",
                          reg_alpha=0, reg_lambda=1, scale_pos_weight=33, subsample=0.2, 
                          nthread=6, random_state=SEED).fit(X_train_features, y_train)

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[13:50:01] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [11]:
explainer = ClassifierExplainer(model, X_test_features, y_test,
                               shap='tree',
                               model_output='probability')

Detected XGBClassifier model: Changing class type to XGBClassifierExplainer...
Note: model_output=='probability'. For XGBClassifier shap values normally get calculated against X_background, but paramater X_background=None, so using X instead
Generating self.shap_explainer = shap.TreeExplainer(model, X, model_output='probability', feature_perturbation='interventional')...
Note: Shap interaction values will not be available. If shap values in probability space are not necessary you can pass model_output='logodds' to get shap values in logodds without the need for a background dataset and also working shap interaction values...


In [ ]:
ExplainerDashboard(explainer,
                   importances=True,
                   model_summary=True,
                   contributions=True,
                   whatif=True,
                   shap_dependence=True,
                   shap_interaction=True,
                   decision_trees=True
                  ).run(port=8051)

Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
For this type of model and model_output interactions don't work, so setting shap_interaction=False...
Generating layout...
Calculating shap values...


100%|===================| 42128/42173 [06:18<00:00]        

Generating xgboost model dump...
Calculating dependencies...
Calculating permutation importances (if slow, try setting n_jobs parameter)...
Calculating roc auc curves...
Calculating prediction probabilities...
Calculating pred_percentiles...
Calculating pr auc curves...
Calculating liftcurve_dfs...
Calculating confusion matrices...
Calculating classification_dfs...
Calculating predictions...
Calculating metrics...
Calculating ShadowDecTree for each individual decision tree...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Starting ExplainerDashboard on http://192.168.0.223:8051
Dash is running on http://0.0.0.0:8051/

 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8051/ (Press CTRL+C to quit)
192.168.0.1 - - [29/Jun/2021 14:03:09] "GET / HTTP/1.1" 200 -
192.168.0.1 - - [29/Jun/2021 14:03:09] "GET /_dash-component-suites/dash_renderer/react@16.v1_9_1m1617985068.14.0.min.js HTTP/1.1" 200 -
192.168.0.1 - - [29/Jun/2021 14:03:09] "GET /assets/bootstrap.min.css?m=1616526662.0 HTTP/1.1" 200 -
192.168.0.1 - - [29/Jun/2021 14:03:09] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_9_1m1617985068.7.2.min.js HTTP/1.1" 200 -
192.168.0.1 - - [29/Jun/2021 14:03:09] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_9_1m1617985068.8.7.min.js HTTP/1.1" 200 -
192.168.0.1 - - [29/Jun/2021 14:03:09] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_9_1m1617985068.14.0.min.js HTTP/1.1" 200 -
192.168.0.1 - - [29/Jun/2021 14:03:09] "GET /_dash-component-suites/dash_table/bundle.v4_11_3m1617985106.js HTTP/1.1" 200 -
192.168.0.1 - - [29/Jun/2021 14:03:09] "GET /_dash-component-suites/dash_bootstrap_components/_com